<a href="https://colab.research.google.com/github/jmcgonegal/colab/blob/master/BatchRunImagenetClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Convolution2D, MaxPooling2D, Activation, BatchNormalization, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

filepath="weights.best.hdf5"
fill_mode='nearest'

train_data_dir = 'train' # we are just doing this to be lazy about getting a list of classes
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    fill_mode=fill_mode)

# path to the model weights files.
# dimensions of our images.
img_width, img_height = 384, 384

batch_size = 1 # how many images to train in the batch (limited by size of memory)
class_mode='categorical'
    
# images we will use for validation
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode=class_mode)

# our image shape width,height,rgb
input_shape = (img_width, img_height, 3)

# build the VGG16/VGG19 network
vgg_model = applications.VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

# dont adjust weights of these layers
for i in range(len(vgg_model.layers)):
    vgg_model.layers[i].trainable = False
    
# build our training model
top_model = Sequential()
top_model.add(vgg_model)
top_model.add(Flatten(input_shape=input_shape))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='sigmoid'))

top_model.summary()

# todo check to make sure file exists
top_model.load_weights(filepath)

# compile the model
top_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=1e-5), 
                  metrics=['accuracy'])

print('Model loaded')


In [0]:
import os
import shutil

labels = (validation_generator.class_indices) 
labels = dict((v,k) for k,v in labels.items()) # reverse key and value for label lookup

unclass_gen = ImageDataGenerator(
    rescale=1. / 255,
    fill_mode=fill_mode)

unclassified_generator = unclass_gen.flow_from_directory(
    'unsorted',
    target_size=(img_height, img_width),
    batch_size=1,
    color_mode='rgb',
    class_mode=None,
    shuffle=False)

# make the directory we will copy our sorted images to
destPath = os.path.join('classified')
if not os.path.exists(destPath):
    os.mkdir(destPath)

count=0 # we are not shuffled
for gen in unclassified_generator:
    # loop until we have viewed every file
    if count >= len(unclassified_generator.filenames):
        break
    for ix, img in enumerate(gen):
        
        filename = unclassified_generator.filenames[count]
        count+=1
        
        ximg = img.reshape((1, img.shape[0], img.shape[1], img.shape[2] ))

        
        classes = top_model.predict_classes(ximg)

        label = labels[classes[ix]]

        src = os.path.realpath(os.path.join('unsorted',filename))
        dest = os.path.realpath(os.path.join('classified',label))
        print(src,'->',dest+'/')
        
        
        #os.rename(os.path.join('unsorted',filename), to_copy)
        try:
            if not os.path.exists(dest):
                os.mkdir(dest)
            shutil.copy(src, dest)
        except:
            pass